<div style="text-align: center"><img width=300px src="https://www.uni-koblenz-landau.de/de/koblenz/logo.png"></div>


# VTNA



In [384]:
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import seaborn as sns
import matplotlib.animation as animation
import time
import fileupload

from pylab import *
from IPython.display import display, HTML, FileLink, FileLinks
from ipywidgets import Button, HBox, VBox
from IPython.display import clear_output

import io

import zipfile


In [385]:
#Install extentions:
#https://github.com/peteut/ipython-file-upload
#https://github.com/ipython-contrib/jupyter_contrib_nbextensions

network_data = None
attribute_data = None

testfile = None

graph = None
fig_num = None
ht9_aggr_layout = None
ht9_tem_agg_net = None

max_filesize_import = 5000000

update_delta_sec = 20

example_network_data = 'data/dummy_network.txt'
example_attribute_data = 'data/dummy_attributes.txt'


In [386]:
time_slider = widgets.IntSlider(
    #value=initial_time,
    #min=initial_time,
    description='Timeline: ',
    #max=ht9_df['timestamp'].max(),
    step=update_delta_sec,
    continuous_update=False
)

play = widgets.Play(
#     interval=10,
    value=50,
    min=0,
    #max=ht9_df['timestamp'].max(),
    step=20,
    description="Press play",
    disabled=False
)

In [387]:
def safe_div(x,y):
    if y == 0:
        return 0
    return x / y


def create_greaph():
    global testfile
    global fig_num
    global ht9_aggr_layout
    global ht9_tem_agg_net
    ht9_df = pd.read_csv(testfile, sep='\t', header=None, names=['timestamp', 'node1', 'node2'])
    time_sec = ht9_df['timestamp'].max() - ht9_df['timestamp'].min()


    


    ht9_aggr_df = ht9_df.groupby(['node1', 'node2'], as_index=False).count().rename(columns={'timestamp': 'observations'})
    ht9_aggr_net = nx.Graph()
    ht9_aggr_net.add_edges_from((t[1], t[2], {'observations': t[3]}) for t in ht9_aggr_df.itertuples())
    ht9_aggr_deg_cent = list(nx.degree_centrality(ht9_aggr_net).values())
    ht9_aggr_clo_cent = list(nx.closeness_centrality(ht9_aggr_net).values())
    ht9_aggr_layout = nx.spring_layout(ht9_aggr_net, k=2.0/np.sqrt(len(ht9_aggr_net.nodes)))


    # Compute layout of nodes for aggregated network.
    ht9_aggr_layout = nx.spring_layout(ht9_aggr_net, k=safe_div(2.0,np.sqrt(len(ht9_aggr_net.nodes))))
    initial_time = ht9_df['timestamp'].min()
    time_slider.value = 0
    time_slider.min = 0
    time_slider.max=ht9_df['timestamp'].max()
    play.max=ht9_df['timestamp'].max()
    plt.figure(figsize=(8, 8))
    fig = plt.gcf()
    axes = plt.gca()
    fig_num = fig.number
    global graph
    graph = ht9_df


def draw_current_temp_aggr(current_time):
    global graph
    plt.figure(fig_num, figsize=(8, 8))
    fig = plt.gcf()
    axes = plt.gca()
    axes.set_xlim((-1.2, 1.2))
    axes.set_ylim((-1.2, 1.2))
    if graph is not None and fig_num is not None:
        ht9_tem_agg_net = nx.Graph()
        ht9_tem_agg_net.add_edges_from([(t[2], t[3]) for t in graph[graph['timestamp'] == current_time].itertuples()])
        nx.draw_networkx(ht9_tem_agg_net, ht9_aggr_layout, ax=axes, with_labels=False, node_size=75, node_color='green')
    axes.set_title('time = {}'.format(current_time))
    plt.show()
    
    
    




In [388]:



#widgets.interact(draw_current_temp_aggr, current_time=time_slider)
plot = widgets.interactive(draw_current_temp_aggr, current_time=time_slider)



widgets.jslink((play, 'value'), (time_slider, 'value'))
playbox = widgets.HBox([play,time_slider])
graphbox = widgets.VBox([plot, playbox])


In [389]:
def on_toogle_filetype(change):
    if change['new'] == 'Network':
        w_upload_local_nf.layout.display = 'none'
        w_upload_local_af.layout.display = 'none'

        w_upload_network_nf.layout.display = 'inline'
        w_upload_network_af.layout.display = 'inline'
        
        w_attribute_file.disabled = False
        w_attribute_file.placeholder='www.url_to_file.de (Allowed types: .zip, .txt ; Max filesize = {:.2f} mB)'.format(max_filesize_import/10**6)
        w_network_file.disabled = False
        w_network_file.placeholder='www.url_to_file.de (Allowed types: .zip, .txt ; Max filesize = {:.2f} mB)'.format(max_filesize_import/10**6)

    else:  
        w_upload_local_nf.layout.display = 'inline'
        w_upload_local_af.layout.display = 'inline'

        w_upload_network_nf.layout.display = 'none'
        w_upload_network_af.layout.display = 'none'
        
        w_attribute_file.disabled = True
        w_attribute_file.placeholder='Allowed types: .zip, .txt ; Max filesize = 5mb'
        w_network_file.disabled = True
        w_network_file.placeholder='Allowed types: .zip, .txt ; Max filesize = 5mb'

def _handle_upload_nf(change):
    global testfile
    w = change['owner']    
    with open(w.filename, 'wb') as f:
        if ((w.filename.endswith('.dat') or w.filename.endswith('.txt') or w.filename.endswith('.zip')) and len(w.data) < max_filesize_import) :
            f.write(w.data)
            w_network_file.value = 'Uploaded `{}` ({:.2f} kB)'.format(w.filename, len(w.data) / 2**10)
        else:
            w_network_file.value = 'Invalid filetype or size!'
            return
    network_data=pd.read_table(w.filename, header=None, delim_whitespace=True, names=['timestamp', 'node1', 'node2'])
    testfile = w.filename
    
    with w_out_network:
        clear_output()
        print('Preview {} :'.format(w.filename))
        print(display(network_data.head(4)))
        
def _handle_upload_af(change):
    w = change['owner']      
    with open(w.filename, 'wb') as f:   
        if ((w.filename.endswith('.dat') or w.filename.endswith('.txt') or w.filename.endswith('.zip')) and len(w.data) < max_filesize_import) :
            f.write(w.data)           
            w_attribute_file.value = 'Uploaded `{}` ({:.2f} kB)'.format(w.filename, len(w.data) / 2**10)
        else:
            w_attribute_file.value = 'Invalid filetype or size!'
            return
    attribute_data=pd.read_table(w.filename, header=None, delim_whitespace=True)
    with w_out_attributes:
        clear_output()
        print('Preview {} :'.format(w.filename))
        print(display(attribute_data.head(4)))
    open_colum_config(attribute_data)    
        
def _handle_network_upload_nf(change):    
    url = w_network_file.value#'http://www.sociopatterns.org/wp-content/uploads/2016/06/tij_InVS.dat_.zip'
    network_data=pd.read_table(url, header=None, delim_whitespace=True)
    with w_out_network:
        clear_output()
        print('Preview {} :'.format(url))
        print(display(network_data.head(4)))
    
        
def _handle_network_upload_af(change):    
    url = w_attribute_file.value#'http://www.sociopatterns.org/wp-content/uploads/2016/06/tij_InVS.dat_.zip'
    attribute_data=pd.read_table(url, header=None, delim_whitespace=True)
    with w_out_attributes:
        clear_output()
        print('Preview {} :'.format(url))
        print(display(attribute_data.head(4)))
        print(len(attribute_data.columns))
    open_colum_config(attribute_data)
        
def open_colum_config(attribute_data):
    print(len(attribute_data.columns))
    

In [390]:
box_layout = widgets.Layout(border='solid 1px rgb(210,210,210)',margin="5px 5px 5px 5px")
box_layout_ = widgets.Layout(margin="10px 0px 10px 0px")
simbox_layout = widgets.Layout(border='solid 1px rgb(210,210,210)',margin="5px 5px 5px 5px",width="100%",height="600px")
upload_layout = widgets.Layout(width="100%")


w_upload_type = widgets.ToggleButtons(
    options=['Computer', 'Network'],
    description='Source:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Upload file from local folder', 'Upload from network path'],

)


w_upload_type.observe(on_toogle_filetype, 'value')

w_network_file = widgets.Text(
    value='',
    placeholder='Allowed types: .zip, .txt ; Max filesize = 5mb',
    description='Network file:',
    disabled=True,
    layout=upload_layout
)


w_attribute_file = widgets.Text(
    value='',
    placeholder='Allowed types: .zip, .txt ; Max filesize = 5mb',
    description='Attribute file:',
    disabled=True, 
    layout=upload_layout
)

w_upload_local_nf = fileupload.FileUploadWidget(    
    label="Upload",
)
w_upload_local_nf.observe(_handle_upload_nf, names='data')


w_upload_local_af = fileupload.FileUploadWidget(    
    label="Upload",
)
w_upload_local_af.observe(_handle_upload_af, names='data')


w_upload_network_nf = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='info',
    tooltip='Upload network file',
    icon='upload'
)
w_upload_network_nf.layout.display = 'none'
w_upload_network_nf.on_click(_handle_network_upload_nf)

w_upload_network_af = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='info', 
    tooltip='Upload attribute file',
    icon='upload'
)
w_upload_network_af.layout.display = 'none'
w_upload_network_af.on_click(_handle_network_upload_af)


w_out_network = widgets.Output()
network_data=pd.read_table(example_network_data, header=None, delim_whitespace=True)

with w_out_network:
    print("Please upload tab-separated values, see this example:")
    print(display(network_data))

w_out_attributes = widgets.Output()
attribute_data=pd.read_table(example_attribute_data, header=None, delim_whitespace=True)
with w_out_attributes:
    print("Please upload tab-separated values, see this example:")
    print(display(attribute_data))
    



w_run = widgets.Button(
    description='Run',
    disabled=False,
    button_style='success',
    tooltip='Run simulation',
    icon='start'
)
w_import = widgets.Button(
    description='Import',
    disabled=False,
    button_style='warning',
    tooltip='Import',
    icon='start'
)
w_progress=widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    description='Loading:',
    bar_style='info',
    orientation='horizontal'
)
w_progress.layout.visibility ='hidden'
w_upload_typeHBox=widgets.HBox([w_upload_type])

w_network_uploadHBox=widgets.HBox([w_network_file, w_upload_local_nf, w_upload_network_nf])


#w_network_settings_left = widgets.VBox([w_timestamp_dropdown, w_node01_dropdown,w_node02_dropdown])
w_network_settings=widgets.HBox([w_out_network])

import_network_box = widgets.VBox([w_upload_typeHBox,w_network_uploadHBox,w_network_settings],layout=box_layout)

w_attribute_uploadHBox=widgets.HBox([w_attribute_file, w_upload_local_af, w_upload_network_af])

w_col01_attribute = widgets.VBox([w_col01_name,w_type_dropdown],layout=box_layout_)
w_col02_attribute = widgets.VBox([w_col02_name,w_type_dropdown],layout=box_layout_)
w_col03_attribute = widgets.VBox([w_col03_name,w_type_dropdown],layout=box_layout_)



w_attribute_settings_left = widgets.VBox([])#widgets.VBox([w_nodeID_dropdown, w_col01_attribute,w_col02_attribute,w_col03_attribute])

w_attribute_settings=widgets.HBox([ w_attribute_settings_left, w_out_attributes ])
import_attribute_box = widgets.VBox([w_attribute_uploadHBox,w_attribute_settings],layout=box_layout)

w_toolbar=widgets.HBox([w_run, w_progress])
import_box = widgets.VBox([import_network_box,import_attribute_box,w_toolbar])

simulation_box = widgets.VBox([w_import,  graphbox],layout=simbox_layout)
simulation_box.layout.display = 'none'

def on_run(b):
    create_greaph()
    w_progress.layout.visibility ='visible'
    #for i in range(10):
    #    time.sleep(0.3)
    #   w_progress.value+=1
    import_box.layout.display = 'none'
    
    simulation_box.layout.display = 'block'
    
def on_import(b):
    simulation_box.layout.display = 'none'
    w_progress.value=0
    w_progress.layout.visibility ='hidden'
    import_box.layout.display = 'block'
        
w_run.on_click(on_run)
w_import.on_click(on_import)
display(simulation_box)
display(import_box)

VBox(children=(Button(button_style='warning', description='Import', icon='start', style=ButtonStyle(), tooltip='Import'), VBox(children=(interactive(children=(IntSlider(value=0, continuous_update=False, description='Timeline: ', step=20), Output()), _dom_classes=('widget-interact',)), HBox(children=(Play(value=50, description='Press play', step=20), IntSlider(value=0, continuous_update=False, description='Timeline: ', step=20)))))), layout=Layout(border='solid 1px rgb(210,210,210)', display='none', height='600px', margin='5px 5px 5px 5px', width='100%'))

VBox(children=(VBox(children=(HBox(children=(ToggleButtons(button_style='info', description='Source:', options=('Computer', 'Network'), tooltips=['Upload file from local folder', 'Upload from network path'], value='Computer'),)), HBox(children=(Text(value='', description='Network file:', disabled=True, layout=Layout(width='100%'), placeholder='Allowed types: .zip, .txt ; Max filesize = 5mb'), FileUploadWidget(label='Upload', _dom_classes=('widget_item', 'btn-group')), Button(button_style='info', description='Upload', icon='upload', layout=Layout(display='none'), style=ButtonStyle(), tooltip='Upload network file'))), HBox(children=(Output(),))), layout=Layout(border='solid 1px rgb(210,210,210)', margin='5px 5px 5px 5px')), VBox(children=(HBox(children=(Text(value='', description='Attribute file:', disabled=True, layout=Layout(width='100%'), placeholder='Allowed types: .zip, .txt ; Max filesize = 5mb'), FileUploadWidget(label='Upload', _dom_classes=('widget_item', 'btn-group')), Button(button_style='info', description='Upload', icon='upload', layout=Layout(display='none'), style=ButtonStyle(), tooltip='Upload attribute file'))), HBox(children=(VBox(), Output()))), layout=Layout(border='solid 1px rgb(210,210,210)', margin='5px 5px 5px 5px')), HBox(children=(Button(button_style='success', description='Run', icon='start', style=ButtonStyle(), tooltip='Run simulation'), IntProgress(value=0, bar_style='info', description='Loading:', layout=Layout(visibility='hidden'), max=10)))))

In [391]:

exportbutton = widgets.Button(
    value=False,
    description='Convert',
    disabled=False,
    button_style='', 
    tooltip='Download animation',
    icon='download'
)

In [392]:
filetyperadios = widgets.RadioButtons(
    options=['Mp4', 'Gif'],
    description='Filetype:',
    disabled=False
)

In [393]:
box = HBox([filetyperadios, exportbutton])
settings = HBox([filetyperadios, exportbutton])

In [394]:
tab_nest = widgets.Tab()
tab_nest.children = [box, settings]
tab_nest.set_title(0, 'Export')
tab_nest.set_title(1, 'Settings')
#tab_nest

In [395]:

# To Install ffmpeg: conda install -c conda-forge ffmpeg
'''

dpi = 100


plt.figure(fig_num, figsize=(8, 8))
fig = plt.gcf()
axes = plt.gca()
axes.set_xlim((-1.2, 1.2))
axes.set_ylim((-1.2, 1.2))


def draw_current_temp_aggr(current_time):
    current_time = current_time*20
    fig = plt.gcf()
    ht9_tem_agg_net = nx.Graph()
    ht9_tem_agg_net.add_edges_from([(t[2], t[3]) for t in ht9_df[ht9_df['timestamp'] == current_time].itertuples()])
    axes.set_title('time = {}'.format(current_time))
    nx.draw_networkx(ht9_tem_agg_net, ht9_aggr_layout, ax=axes, with_labels=False, node_size=75, node_color='green')



#ani = animation.FuncAnimation(fig,draw_current_temp_aggr,5000/20,interval=30)
#writer = animation.writers['ffmpeg'](fps=30)

#ani.save('demo.mp4',writer=writer,dpi=dpi)
'''

"\n\ndpi = 100\n\n\nplt.figure(fig_num, figsize=(8, 8))\nfig = plt.gcf()\naxes = plt.gca()\naxes.set_xlim((-1.2, 1.2))\naxes.set_ylim((-1.2, 1.2))\n\n\ndef draw_current_temp_aggr(current_time):\n    current_time = current_time*20\n    fig = plt.gcf()\n    ht9_tem_agg_net = nx.Graph()\n    ht9_tem_agg_net.add_edges_from([(t[2], t[3]) for t in ht9_df[ht9_df['timestamp'] == current_time].itertuples()])\n    axes.set_title('time = {}'.format(current_time))\n    nx.draw_networkx(ht9_tem_agg_net, ht9_aggr_layout, ax=axes, with_labels=False, node_size=75, node_color='green')\n\n\n\n#ani = animation.FuncAnimation(fig,draw_current_temp_aggr,5000/20,interval=30)\n#writer = animation.writers['ffmpeg'](fps=30)\n\n#ani.save('demo.mp4',writer=writer,dpi=dpi)\n"

In [396]:

#FileLinks('.')

In [397]:
#html = '<a download="filename" href="/files/demo1.mp4" target="_blank">Download</a>'
#HTML(html)



In [398]:
# http://chris-said.io/2016/02/13/how-to-make-polished-jupyter-presentations-with-optional-code-visibility/
HTML('''
<script>
    code_show=true; 
    function code_toggle() {
         if (code_show){
             $('div.input').hide();
         } else {
             $('div.input').show();
         }
         code_show = !code_show
    } 
    $( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.
''')